<a href="https://colab.research.google.com/github/Lnsfna/ThesisExperiment/blob/main/thesispracticum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Preparation

##Installation of necessary libraries

In [ ]:
!pip install yolov5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.4/953.4 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.9/756.9 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 20

I've modified the CnOcr package and uploaded it to my Google Drive. At this step i mount my google drive, clone the package and install requirements for it.


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import shutil
shutil.copytree("/content/drive/MyDrive/CnOcr", "./CnOCR")

'./CnOCR'

In [ ]:
! pip install -r /content/CnOCR/requirements.txt

Wandb key should be pasted here to train CnOcr model at the PGD-based adversarial training step

In [ ]:
import wandb
wandb.login(key='KEY HERE')

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

## Import libraries

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import glob
from functools import reduce
from CnOCR.cnocr.cn_ocr import CnOcr
from CnOCR.cnocr.recognizer import gen_model

from torchvision import transforms
from PIL import Image
import torch
import yolov5
import torch.nn as nn
import torch.nn.functional as F
import re
from matplotlib import font_manager
from sklearn.metrics import mean_absolute_percentage_error
import random


import matplotlib.pyplot as plt
%matplotlib inline


##Define constants

In [ ]:
provinces = ["皖", "沪", "津", "渝", "冀", "晋", "蒙", "辽", "吉", "黑", "苏", "浙", "京", "闽", "赣", "鲁", "豫", "鄂", "湘", "粤", "桂", "琼", "川", "贵", "云", "藏", "陕", "甘", "青", "宁", "新", "警", "学", "O"]
alphabets = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'O']
ads = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X',
       'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'O']

list_sub = ["ccpd_base", "ccpd_fn", "ccpd_db", "ccpd_rotate", "ccpd_weather", "ccpd_blur"]
BASE_PATH = "/content/drive/MyDrive/CCPD2019_2K"

#Data preparation

In [ ]:
def extract_plate_number(plate_number):
    chi_let = provinces[int(plate_number.split("_")[0])]
    alp_let = alphabets[int(plate_number.split("_")[1])]
    alp_num_let = plate_number.split("_")[2:]
    alp_num_let = "".join([ads[int(char)] for char in alp_num_let])
    all_let = chi_let + alp_let + alp_num_let
    return all_let

df_metadata = pd.DataFrame()
for sub in list_sub:
    sub_path = os.path.join(BASE_PATH, sub, "*")
    sub_files = glob.glob(sub_path)

    df_metadata = pd.concat([df_metadata, pd.DataFrame(sub_files)])

df_metadata = df_metadata.rename(columns={0: "image_path"}).reset_index(drop=True)
df_metadata["sub"] = df_metadata["image_path"].apply(lambda x: x.split("/")[5])
df_metadata["detail"] = df_metadata["image_path"].apply(lambda x: x.split("/")[6])
df_metadata["bbox"] = df_metadata["detail"].apply(lambda x: x.split("-")[2])
df_metadata["x1_bbox"] = df_metadata["bbox"].apply(lambda x: int(x.split("_")[0].split("&")[0]))
df_metadata["y1_bbox"] = df_metadata["bbox"].apply(lambda x: int(x.split("_")[0].split("&")[1]))
df_metadata["x2_bbox"] = df_metadata["bbox"].apply(lambda x: int(x.split("_")[1].split("&")[0]))
df_metadata["y2_bbox"] = df_metadata["bbox"].apply(lambda x: int(x.split("_")[1].split("&")[1]))
df_metadata["plate_number"] = df_metadata["detail"].apply(lambda x: x.split("-")[4])
df_metadata["plate_number"] = df_metadata["plate_number"].apply(lambda x: extract_plate_number(x))

df_metadata = df_metadata.drop(["detail", "bbox"], axis=1)
df_metadata

,image_path,sub,x1_bbox,y1_bbox,x2_bbox,y2_bbox,plate_number
0,/content/drive/MyDrive/CCPD2019_2K/ccpd_base/0...,ccpd_base,360,479,546,543,皖AKJ718
1,/content/drive/MyDrive/CCPD2019_2K/ccpd_base/0...,ccpd_base,208,474,414,579,皖AWC386
2,/content/drive/MyDrive/CCPD2019_2K/ccpd_base/0...,ccpd_base,283,486,492,550,皖AV611U
3,/content/drive/MyDrive/CCPD2019_2K/ccpd_base/0...,ccpd_base,272,512,481,574,皖AZ9X58
4,/content/drive/MyDrive/CCPD2019_2K/ccpd_base/0...,ccpd_base,280,442,449,515,皖A0N621
...,...,...,...,...,...,...,...
1995,/content/drive/MyDrive/CCPD2019_2K/ccpd_blur/0...,ccpd_blur,306,494,408,531,皖A59S98
1996,/content/drive/MyDrive/CCPD2019_2K/ccpd_blur/0...,ccpd_blur,253,377,353,415,皖CR2366
1997,/content/drive/MyDrive/CCPD2019_2K/ccpd_blur/0...,ccpd_blur,329,554,505,619,皖A2S216
1998,/content/drive/MyDrive/CCPD2019_2K/ccpd_blur/0...,ccpd_blur,239,418,466,519,皖AC8Y96


In [ ]:
df_metadata['xmid'] = (df_metadata['x1_bbox'] + df_metadata['x2_bbox']) / (2*720)
df_metadata['ymid'] = (df_metadata['y1_bbox'] + df_metadata['y2_bbox']) / (2*1160)

df_metadata['bb_width'] = (df_metadata['x2_bbox'] - df_metadata['x1_bbox']) / 720
df_metadata['bb_height'] = (df_metadata['y2_bbox'] - df_metadata['y1_bbox']) / 1160

df_metadata

,image_path,sub,x1_bbox,y1_bbox,x2_bbox,...,plate_number,xmid,ymid,bb_width,bb_height
0,/content/drive/MyDrive/CCPD2019_2K/ccpd_base/0...,ccpd_base,360,479,546,...,皖AKJ718,0.629167,0.440517,0.258333,0.055172
1,/content/drive/MyDrive/CCPD2019_2K/ccpd_base/0...,ccpd_base,208,474,414,...,皖AWC386,0.431944,0.453879,0.286111,0.090517
2,/content/drive/MyDrive/CCPD2019_2K/ccpd_base/0...,ccpd_base,283,486,492,...,皖AV611U,0.538194,0.446552,0.290278,0.055172
3,/content/drive/MyDrive/CCPD2019_2K/ccpd_base/0...,ccpd_base,272,512,481,...,皖AZ9X58,0.522917,0.468103,0.290278,0.053448
4,/content/drive/MyDrive/CCPD2019_2K/ccpd_base/0...,ccpd_base,280,442,449,...,皖A0N621,0.506250,0.412500,0.234722,0.062931
...,...,...,...,...,...,...,...,...,...,...,...
1995,/content/drive/MyDrive/CCPD2019_2K/ccpd_blur/0...,ccpd_blur,306,494,408,...,皖A59S98,0.495833,0.441810,0.141667,0.031897
1996,/content/drive/MyDrive/CCPD2019_2K/ccpd_blur/0...,ccpd_blur,253,377,353,...,皖CR2366,0.420833,0.341379,0.138889,0.032759
1997,/content/drive/MyDrive/CCPD2019_2K/ccpd_blur/0...,ccpd_blur,329,554,505,...,皖A2S216,0.579167,0.505603,0.244444,0.056034
1998,/content/drive/MyDrive/CCPD2019_2K/ccpd_blur/0...,ccpd_blur,239,418,466,...,皖AC8Y96,0.489583,0.403879,0.315278,0.087069


#Licence plate recognition

##Load models

In [ ]:
cnocr_model = CnOcr()


inited model


In [ ]:
model = yolov5.load('keremberke/yolov5m-license-plate')

model.conf = 0.25
model.iou = 0.45
model.agnostic = False
model.multi_label = False
model.max_det = 1


config.json:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

best.pt:   0%|          | 0.00/42.1M [00:00<?, ?B/s]

##LPR Detection

In [ ]:
x1_pred = []
y1_pred = []
x2_pred = []
y2_pred = []
mape = []

for i in range (df_metadata.shape[0]):
  img = df_metadata.at[i, 'image_path']

  results = model(img, size=640)

  predictions = results.pred[0]
  try:
    boxes = predictions[:, :4].tolist()[0] # x1, y1, x2, y2
    x1_pred.append(boxes[0])
    y1_pred.append(boxes[1])
    x2_pred.append(boxes[2])
    y2_pred.append(boxes[3])
    target = [df_metadata.at[i, "x1_bbox"], df_metadata.at[i, "y1_bbox"], df_metadata.at[i, "x2_bbox"], df_metadata.at[i, "y2_bbox"]]
    acc = mean_absolute_percentage_error(target, boxes)
    mape.append(acc)
  except IndexError:
    x1_pred.append(None)
    y1_pred.append(None)
    x2_pred.append(None)
    y2_pred.append(None)
    mape.append(None)

In [ ]:
df_new = df_metadata.copy()
df_new["x1_pred"] = x1_pred
df_new["y1_pred"] = y1_pred
df_new["x2_pred"] = x2_pred
df_new["y2_pred"] = y2_pred
df_new["mape"] = mape

In [ ]:
df_new.to_csv('out.csv', index=False)



In [ ]:
!cp '/content/out.csv' /content/drive/MyDrive/


In [ ]:
df_metadata = pd.read_csv('/content/drive/MyDrive/out.csv')

In [ ]:
df_metadata

,image_path,sub,x1_bbox,y1_bbox,x2_bbox,y2_bbox,plate_number,xmid,ymid,bb_width,bb_height,x1_pred,y1_pred,x2_pred,y2_pred,mape
0,/content/drive/MyDrive/CCPD2019_2K/ccpd_base/0...,ccpd_base,360,479,546,543,皖AKJ718,0.629167,0.440517,0.258333,0.055172,347.686920,470.430634,544.028687,543.437561,0.014127
1,/content/drive/MyDrive/CCPD2019_2K/ccpd_base/0...,ccpd_base,208,474,414,579,皖AWC386,0.431944,0.453879,0.286111,0.090517,189.605759,480.267761,433.662598,574.395508,0.039276
2,/content/drive/MyDrive/CCPD2019_2K/ccpd_base/0...,ccpd_base,283,486,492,550,皖AV611U,0.538194,0.446552,0.290278,0.055172,269.474243,475.081604,489.774628,560.677612,0.023549
3,/content/drive/MyDrive/CCPD2019_2K/ccpd_base/0...,ccpd_base,272,512,481,574,皖AZ9X58,0.522917,0.468103,0.290278,0.053448,267.586731,506.686188,474.200897,578.223267,0.012024
4,/content/drive/MyDrive/CCPD2019_2K/ccpd_base/0...,ccpd_base,280,442,449,515,皖A0N621,0.506250,0.412500,0.234722,0.062931,276.568329,437.475647,449.386658,516.370300,0.006503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,/content/drive/MyDrive/CCPD2019_2K/ccpd_blur/0...,ccpd_blur,306,494,408,531,皖A59S98,0.495833,0.441810,0.141667,0.031897,297.068451,495.310333,409.928680,530.465698,0.009393
1996,/content/drive/MyDrive/CCPD2019_2K/ccpd_blur/0...,ccpd_blur,253,377,353,415,皖CR2366,0.420833,0.341379,0.138889,0.032759,250.468613,378.564392,351.658234,414.848846,0.004580
1997,/content/drive/MyDrive/CCPD2019_2K/ccpd_blur/0...,ccpd_blur,329,554,505,619,皖A2S216,0.579167,0.505603,0.244444,0.056034,564.502869,409.671478,681.245728,462.777710,0.394429
1998,/content/drive/MyDrive/CCPD2019_2K/ccpd_blur/0...,ccpd_blur,239,418,466,519,皖AC8Y96,0.489583,0.403879,0.315278,0.087069,236.132278,428.112366,461.985229,508.324097,0.016344


In [ ]:
def mdape(actual: np.ndarray, predicted: np.ndarray):
    error_percent = (actual - predicted) /(actual + 1e-10)
    return np.median(np.abs(error_percent))

In [ ]:
mdape_list = []
for i in range (df_metadata.shape[0]):
    target = np.array([df_metadata.at[i, "x1_bbox"], df_metadata.at[i, "y1_bbox"], df_metadata.at[i, "x2_bbox"], df_metadata.at[i, "y2_bbox"]])
    predict = np.array([df_metadata.at[i, "x1_pred"], df_metadata.at[i, "y1_pred"], df_metadata.at[i, "x2_pred"], df_metadata.at[i, "y2_pred"]])
    e = mdape(target, predict)
    mdape_list.append(e)



In [ ]:
df_new = df_metadata.copy()
df_new["det_mdape"] = mdape_list

In [ ]:
df_new["det_mdape"].fillna(1.0).sum() / df_new.shape[0] * 100

7.1839312691128825

In [ ]:
df_metadata = df_new.copy()
df_metadata

,image_path,sub,x1_bbox,y1_bbox,x2_bbox,y2_bbox,plate_number,xmid,ymid,bb_width,bb_height,x1_pred,y1_pred,x2_pred,y2_pred,mape,mdape
0,/content/drive/MyDrive/CCPD2019_2K/ccpd_base/0...,ccpd_base,360,479,546,543,皖AKJ718,0.63,0.44,0.26,0.06,347.69,470.43,544.03,543.44,0.01,0.01
1,/content/drive/MyDrive/CCPD2019_2K/ccpd_base/0...,ccpd_base,208,474,414,579,皖AWC386,0.43,0.45,0.29,0.09,189.61,480.27,433.66,574.40,0.04,0.03
2,/content/drive/MyDrive/CCPD2019_2K/ccpd_base/0...,ccpd_base,283,486,492,550,皖AV611U,0.54,0.45,0.29,0.06,269.47,475.08,489.77,560.68,0.02,0.02
3,/content/drive/MyDrive/CCPD2019_2K/ccpd_base/0...,ccpd_base,272,512,481,574,皖AZ9X58,0.52,0.47,0.29,0.05,267.59,506.69,474.20,578.22,0.01,0.01
4,/content/drive/MyDrive/CCPD2019_2K/ccpd_base/0...,ccpd_base,280,442,449,515,皖A0N621,0.51,0.41,0.23,0.06,276.57,437.48,449.39,516.37,0.01,0.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,/content/drive/MyDrive/CCPD2019_2K/ccpd_blur/0...,ccpd_blur,306,494,408,531,皖A59S98,0.50,0.44,0.14,0.03,297.07,495.31,409.93,530.47,0.01,0.00
1996,/content/drive/MyDrive/CCPD2019_2K/ccpd_blur/0...,ccpd_blur,253,377,353,415,皖CR2366,0.42,0.34,0.14,0.03,250.47,378.56,351.66,414.85,0.00,0.00
1997,/content/drive/MyDrive/CCPD2019_2K/ccpd_blur/0...,ccpd_blur,329,554,505,619,皖A2S216,0.58,0.51,0.24,0.06,564.50,409.67,681.25,462.78,0.39,0.30
1998,/content/drive/MyDrive/CCPD2019_2K/ccpd_blur/0...,ccpd_blur,239,418,466,519,皖AC8Y96,0.49,0.40,0.32,0.09,236.13,428.11,461.99,508.32,0.02,0.02


In [ ]:
df_metadata.to_csv('out.csv', index=False)


In [ ]:
!cp '/content/out.csv' /content/drive/MyDrive/


##LPR Recognition

In [ ]:
df_metadata = pd.read_csv('/content/drive/MyDrive/out.csv')

In [ ]:
df_metadata["det_mdape"] = df_metadata["det_mdape"].fillna(1.0)

In [ ]:
number_pred = []
for i in range (df_metadata.shape[0]):
  img_path = df_metadata.at[i, 'image_path']
  img = cv2.imread(img_path)

  predict = [df_metadata.at[i, "x1_pred"], df_metadata.at[i, "y1_pred"], df_metadata.at[i, "x2_pred"], df_metadata.at[i, "y2_pred"]]
  text = ""
  mdape_score = df_metadata.at[i, 'det_mdape']
  if mdape_score != 1.0:
    crop_image = img[int(predict[1]):int(predict[3]), int(predict[0]):int(predict[2])]
    ocr_predict = cnocr_model.ocr(crop_image)
    if len(ocr_predict) > 0:
      text = ocr_predict[0]["text"]
  number_pred.append(text)

In [ ]:
df_new = df_metadata.copy()

In [ ]:
df_new["text_pred"] = number_pred

In [ ]:
df_new.to_csv('out.csv', index=False)


In [ ]:
!cp '/content/out.csv' /content/drive/MyDrive/


In [ ]:
df_metadata = df_new.copy()

In [ ]:
def prepare_txt(txt):
  txt = re.sub(r"[ ·:-]", "", txt)
  txt = txt.replace("O", "0").replace("I", "1")
  return txt.upper()

In [ ]:
text_list = []
for i in range(df_metadata.shape[0]):
    text = df_metadata.at[i, 'text_pred']
    if text != "":
      clean_text = prepare_txt(text)
      text_list.append(clean_text)
    else:
      text_list.append(text)



In [ ]:
df_new = df_metadata.copy()
df_new["text_pred"] = text_list

In [ ]:
df_new.to_csv('out.csv', index=False)


In [ ]:
!cp '/content/out.csv' /content/drive/MyDrive/


In [ ]:
df_metadata = df_new.copy()
df_metadata

In [ ]:
df_metadata = pd.read_csv('/content/drive/MyDrive/out.csv')
df_metadata

,image_path,sub,x1_bbox,y1_bbox,x2_bbox,...,mape,det_mdape,text_pred,grad,rec_mdape
0,/content/drive/MyDrive/CCPD2019_2K/ccpd_base/0...,ccpd_base,360,479,546,...,0.014127,0.010750,皖AKJ78,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0]",0.000000
1,/content/drive/MyDrive/CCPD2019_2K/ccpd_base/0...,ccpd_base,208,474,414,...,0.039276,0.030359,EANC386,"[-1.0, 0.0, -1.0, 0.0, 0.0, 0.0, 0.0]",0.000000
2,/content/drive/MyDrive/CCPD2019_2K/ccpd_base/0...,ccpd_base,283,486,492,...,0.023549,0.020940,皖AV6T1U,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]",0.000000
3,/content/drive/MyDrive/CCPD2019_2K/ccpd_base/0...,ccpd_base,272,512,481,...,0.012024,0.012257,诚A79X58,"[-1.0, 0.0, -1.0, 0.0, 0.0, 0.0, 0.0]",0.000000
4,/content/drive/MyDrive/CCPD2019_2K/ccpd_base/0...,ccpd_base,280,442,449,...,0.006503,0.006448,房AM62,"[-1.0, 0.0, 1.0, -1.0, -1.0, 1.0, 1.0]",0.913395
...,...,...,...,...,...,...,...,...,...,...,...
1995,/content/drive/MyDrive/CCPD2019_2K/ccpd_blur/0...,ccpd_blur,306,494,408,...,0.009393,0.003690,1A59590,"[-1.0, 0.0, 0.0, 0.0, -1.0, 0.0, -1.0]",0.000000
1996,/content/drive/MyDrive/CCPD2019_2K/ccpd_blur/0...,ccpd_blur,253,377,353,...,0.004580,0.003975,CR236,"[-1.0, 1.0, -1.0, 1.0, 1.0, 1.0, 1.0]",1.126190
1997,/content/drive/MyDrive/CCPD2019_2K/ccpd_blur/0...,ccpd_blur,329,554,505,...,0.394429,0.304761,一,"[-1.0, -1.0, 1.0, -1.0, 1.0, 1.0, 1.0]",1.720000
1998,/content/drive/MyDrive/CCPD2019_2K/ccpd_blur/0...,ccpd_blur,239,418,466,...,0.016344,0.016284,NaN,"[-1.0, -1.0, -1.0, 1.0, -1.0, 1.0, 1.0]",0.940229


In [ ]:
import random

ocr_model = gen_model('densenet_lite_136-gru', cnocr_model.rec_model._vocab)
loss = nn.MSELoss(reduction='mean')
grad_list = []
mdape_rec = []
df_metadata["text_pred"] = df_metadata["text_pred"].fillna("")

for i in range(df_metadata.shape[0]):
    text_pred = df_metadata.at[i, 'text_pred']
    text_target = df_metadata.at[i, 'plate_number']
    if len(text_pred) < 7:
      while len(text_pred) != 7:
          text_pred += "-"
    if len(text_pred) > 7:
      while len(text_pred) != 7:
        ind = random.randint(0, len(text_pred) - 1)
        text_pred = text_pred[:ind] + text_pred[ind + 1:]

    pred = ocr_model.compute_target([text_pred])
    target = ocr_model.compute_target([text_target])
    predictTensor = torch.tensor(pred[0].astype(float).tolist(), requires_grad=True)
    targetTensor = torch.tensor(target[0].astype(float).tolist(), requires_grad=False)
    tlist = targetTensor.tolist()[0]
    plist = predictTensor.tolist()[0]

    mdape_val = mdape(np.array(tlist), np.array(plist))
    mdape_rec.append(mdape_val)
    loss_res = loss(predictTensor, targetTensor)

    loss_res.backward()
    grad = predictTensor.grad.sign()
    grad_list.append(grad)


In [ ]:
df_new = df_metadata.copy()


In [ ]:
grads = []
for i in grad_list:
  grads.append(i.tolist()[0])

In [ ]:
df_new["grad"] = grads
df_new["rec_mdape"] = mdape_rec

In [ ]:
df_new["rec_mdape"].fillna(1.0).sum() / df_new.shape[0] * 100

44.92105335001856

In [ ]:
df_new.to_csv('out.csv', index=False)


In [ ]:
!cp '/content/out.csv' /content/drive/MyDrive/


In [ ]:
df_metadata = df_new.copy()

##Attacks generation

##FGSM

In [ ]:
df_metadata = pd.read_csv('/content/drive/MyDrive/out.csv')
df_metadata

,image_path,sub,x1_bbox,y1_bbox,x2_bbox,...,mape,det_mdape,text_pred,grad,rec_mdape
0,/content/drive/MyDrive/CCPD2019_2K/ccpd_base/0...,ccpd_base,360,479,546,...,0.014127,0.010750,皖AKJ78,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0]",0.000000
1,/content/drive/MyDrive/CCPD2019_2K/ccpd_base/0...,ccpd_base,208,474,414,...,0.039276,0.030359,EANC386,"[-1.0, 0.0, -1.0, 0.0, 0.0, 0.0, 0.0]",0.000000
2,/content/drive/MyDrive/CCPD2019_2K/ccpd_base/0...,ccpd_base,283,486,492,...,0.023549,0.020940,皖AV6T1U,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]",0.000000
3,/content/drive/MyDrive/CCPD2019_2K/ccpd_base/0...,ccpd_base,272,512,481,...,0.012024,0.012257,诚A79X58,"[-1.0, 0.0, -1.0, 0.0, 0.0, 0.0, 0.0]",0.000000
4,/content/drive/MyDrive/CCPD2019_2K/ccpd_base/0...,ccpd_base,280,442,449,...,0.006503,0.006448,房AM62,"[-1.0, 0.0, 1.0, -1.0, -1.0, 1.0, 1.0]",0.913395
...,...,...,...,...,...,...,...,...,...,...,...
1995,/content/drive/MyDrive/CCPD2019_2K/ccpd_blur/0...,ccpd_blur,306,494,408,...,0.009393,0.003690,1A59590,"[-1.0, 0.0, 0.0, 0.0, -1.0, 0.0, -1.0]",0.000000
1996,/content/drive/MyDrive/CCPD2019_2K/ccpd_blur/0...,ccpd_blur,253,377,353,...,0.004580,0.003975,CR236,"[-1.0, 1.0, -1.0, 1.0, 1.0, 1.0, 1.0]",1.126190
1997,/content/drive/MyDrive/CCPD2019_2K/ccpd_blur/0...,ccpd_blur,329,554,505,...,0.394429,0.304761,一,"[-1.0, -1.0, 1.0, -1.0, 1.0, 1.0, 1.0]",1.720000
1998,/content/drive/MyDrive/CCPD2019_2K/ccpd_blur/0...,ccpd_blur,239,418,466,...,0.016344,0.016284,NaN,"[-1.0, -1.0, -1.0, 1.0, -1.0, 1.0, 1.0]",0.940229


In [ ]:
df_fgsm = df_metadata[["image_path", "x1_bbox", "y1_bbox", "x2_bbox", "y2_bbox", "plate_number"]].copy()
df_fgsm

,image_path,x1_bbox,y1_bbox,x2_bbox,y2_bbox,plate_number
0,/content/drive/MyDrive/CCPD2019_2K/ccpd_base/0...,360,479,546,543,皖AKJ718
1,/content/drive/MyDrive/CCPD2019_2K/ccpd_base/0...,208,474,414,579,皖AWC386
2,/content/drive/MyDrive/CCPD2019_2K/ccpd_base/0...,283,486,492,550,皖AV611U
3,/content/drive/MyDrive/CCPD2019_2K/ccpd_base/0...,272,512,481,574,皖AZ9X58
4,/content/drive/MyDrive/CCPD2019_2K/ccpd_base/0...,280,442,449,515,皖A0N621
...,...,...,...,...,...,...
1995,/content/drive/MyDrive/CCPD2019_2K/ccpd_blur/0...,306,494,408,531,皖A59S98
1996,/content/drive/MyDrive/CCPD2019_2K/ccpd_blur/0...,253,377,353,415,皖CR2366
1997,/content/drive/MyDrive/CCPD2019_2K/ccpd_blur/0...,329,554,505,619,皖A2S216
1998,/content/drive/MyDrive/CCPD2019_2K/ccpd_blur/0...,239,418,466,519,皖AC8Y96


In [ ]:
 def fgsm(img, grad, epsilon):
  grad_shape = grad.shape[0]
  grad_new = grad.view(1, 1, grad_shape).expand(img.shape[0], img.shape[1], grad_shape)
  grad_new = interpolate(grad_new, 3)
  perturbed_img = img + (grad_new * epsilon)
  perturbed_img = perturbed_img.numpy()
  perturbed_img = (perturbed_img * 255).astype(np.uint8)
  perturbed_img = (perturbed_img * 255).astype(np.uint8)
  return perturbed_img

In [ ]:
x1_pred = []
y1_pred = []
x2_pred = []
y2_pred = []
epsilon = 0.02
for i in range(df_metadata.shape[0]):
  path = df_metadata.at[i, 'image_path']
  ori_image = cv2.imread(path)
  predict = [df_metadata.at[i, "x1_bbox"], df_metadata.at[i, "y1_bbox"], df_metadata.at[i, "x2_bbox"], df_metadata.at[i, "y2_bbox"]]
  mdape_score = df_metadata.at[i, 'det_mdape']
  img = torch.from_numpy(ori_image)
  if mdape_score != 1.0:
    crop_image = img[int(predict[1]):int(predict[3]), int(predict[0]):int(predict[2])]
    grad = df_metadata.at[i, 'grad'][1:-2].split(",")
    grad = [float(i) for i in grad]
    grad = torch.tensor(grad)
    pertubed_img = fgsm(crop_image, grad, epsilon)
    img = img.numpy()

    img[int(predict[1]):int(predict[3]), int(predict[0]):int(predict[2])] = pertubed_img
    results = model(img, size=640)
    results = model(img, augment=True)
    predictions = results.pred[0]

    try:
      boxes = predictions[:, :4].tolist()[0] # x1, y1, x2, y2
      x1_pred.append(boxes[0])
      y1_pred.append(boxes[1])
      x2_pred.append(boxes[2])
      y2_pred.append(boxes[3])

    except IndexError:
      x1_pred.append(None)
      y1_pred.append(None)
      x2_pred.append(None)
      y2_pred.append(None)

  else:
    x1_pred.append(None)
    y1_pred.append(None)
    x2_pred.append(None)
    y2_pred.append(None)


In [ ]:
df_fgsm["x1_pred"] = x1_pred
df_fgsm["y1_pred"] = y1_pred
df_fgsm["x2_pred"] = x2_pred
df_fgsm["y2_pred"] = y2_pred

In [ ]:
df_fgsm.to_csv('fgsm_out.csv', index=False)


In [ ]:
!cp '/content/fgsm_out.csv' /content/drive/MyDrive/


In [ ]:
df_fgsm = pd.read_csv('/content/drive/MyDrive/fgsm_out.csv')
df_fgsm


,image_path,x1_bbox,y1_bbox,x2_bbox,y2_bbox,plate_number,x1_pred,y1_pred,x2_pred,y2_pred
0,/content/drive/MyDrive/CCPD2019_2K/ccpd_base/0...,360,479,546,543,皖AKJ718,344.272797,469.219788,546.367737,549.200867
1,/content/drive/MyDrive/CCPD2019_2K/ccpd_base/0...,208,474,414,579,皖AWC386,196.081131,482.628693,430.810364,573.510803
2,/content/drive/MyDrive/CCPD2019_2K/ccpd_base/0...,283,486,492,550,皖AV611U,271.625549,477.087463,487.346497,562.934326
3,/content/drive/MyDrive/CCPD2019_2K/ccpd_base/0...,272,512,481,574,皖AZ9X58,266.222290,505.025085,475.589355,581.188904
4,/content/drive/MyDrive/CCPD2019_2K/ccpd_base/0...,280,442,449,515,皖A0N621,277.394379,439.966400,446.938843,514.871277
...,...,...,...,...,...,...,...,...,...,...
1995,/content/drive/MyDrive/CCPD2019_2K/ccpd_blur/0...,306,494,408,531,皖A59S98,297.779388,492.651794,410.094391,534.257996
1996,/content/drive/MyDrive/CCPD2019_2K/ccpd_blur/0...,253,377,353,415,皖CR2366,251.807297,377.889893,354.802856,416.990509
1997,/content/drive/MyDrive/CCPD2019_2K/ccpd_blur/0...,329,554,505,619,皖A2S216,325.945160,555.169067,501.797150,621.602783
1998,/content/drive/MyDrive/CCPD2019_2K/ccpd_blur/0...,239,418,466,519,皖AC8Y96,234.982986,427.479950,463.455841,502.379883


In [ ]:
path = '/content/drive/MyDrive/CCPD_FGSM'
os.mkdir(path)
for i in list_sub:
  os.mkdir(f"{path}/{i}")

In [ ]:
from PIL import Image
paths = []

for i in range(df_metadata.shape[0]):
  path = df_metadata.at[i, 'image_path']
  ori_image = cv2.imread(path)
  predict = [df_metadata.at[i, "x1_bbox"], df_metadata.at[i, "y1_bbox"], df_metadata.at[i, "x2_bbox"], df_metadata.at[i, "y2_bbox"]]
  mdape_score = df_metadata.at[i, 'det_mdape']
  img = torch.from_numpy(ori_image)
  if mdape_score != 1.0:
    crop_image = img[int(predict[1]):int(predict[3]), int(predict[0]):int(predict[2])]
    grad = df_metadata.at[i, 'grad'][1:-2].split(",")
    grad = [float(i) for i in grad]
    grad = torch.tensor(grad)
    pertubed_img = fgsm(crop_image, grad, epsilon)
    img = img.numpy()

    img[int(predict[1]):int(predict[3]), int(predict[0]):int(predict[2])] = pertubed_img
  if torch.is_tensor(img):
    img = img.numpy()

  img = Image.fromarray(img)
  path = path.replace("CCPD2019_2K", "CCPD_FGSM")
  paths.append(path)
  img.save(path)

In [ ]:
df_fgsm["image_path"] = paths

In [ ]:
x1_pred = []
y1_pred = []
x2_pred = []
y2_pred = []

for i in range (df_metadata.shape[0]):
  img = df_metadata.at[i, 'image_path']

  results = model(img, size=640)
  predictions = results.pred[0]
  try:
    boxes = predictions[:, :4].tolist()[0] # x1, y1, x2, y2
    x1_pred.append(boxes[0])
    y1_pred.append(boxes[1])
    x2_pred.append(boxes[2])
    y2_pred.append(boxes[3])
    target = [df_metadata.at[i, "x1_bbox"], df_metadata.at[i, "y1_bbox"], df_metadata.at[i, "x2_bbox"], df_metadata.at[i, "y2_bbox"]]
  except IndexError:
    x1_pred.append(None)
    y1_pred.append(None)
    x2_pred.append(None)
    y2_pred.append(None)


In [ ]:
mdape_list = []
for i in range (df_fgsm.shape[0]):
    target = np.array([df_fgsm.at[i, "x1_bbox"], df_fgsm.at[i, "y1_bbox"], df_fgsm.at[i, "x2_bbox"], df_fgsm.at[i, "y2_bbox"]])
    predict = np.array([df_fgsm.at[i, "x1_pred"], df_fgsm.at[i, "y1_pred"], df_fgsm.at[i, "x2_pred"], df_fgsm.at[i, "y2_pred"]])
    e = mdape(target, predict)
    mdape_list.append(e)

In [ ]:
df_fgsm["det_mdape"] = mdape_list
df_fgsm["det_mdape"] = df_fgsm["det_mdape"].fillna(1.0)

In [ ]:
df_fgsm["det_mdape"].sum() / df_fgsm.shape[0] * 100

7.958966323596909

In [ ]:
number_pred = []
for i in range (df_fgsm.shape[0]):
  img_path = df_fgsm.at[i, 'image_path']
  img = cv2.imread(img_path)

  predict = [df_fgsm.at[i, "x1_pred"], df_fgsm.at[i, "y1_pred"], df_fgsm.at[i, "x2_pred"], df_fgsm.at[i, "y2_pred"]]
  text = ""
  mdape_score = df_fgsm.at[i, 'det_mdape']
  if mdape_score != 1.0:
    crop_image = img[int(predict[1]):int(predict[3]), int(predict[0]):int(predict[2])]
    ocr_predict = cnocr_model.ocr(crop_image)
    if len(ocr_predict) > 0:
      text = ocr_predict[0]["text"]
  number_pred.append(text)

In [ ]:
df_fgsm["text_pred"] = number_pred
df_fgsm.to_csv('fgsm_out.csv', index=False)


In [ ]:
!cp '/content/fgsm_out.csv' /content/drive/MyDrive/


In [ ]:
text_list = []
for i in range(df_fgsm.shape[0]):
    text = df_fgsm.at[i, 'text_pred']
    if text != "":
      clean_text = re.sub(r"[ ·:-]", "", text)
      clean_text = clean_text.replace("O", "0")
      clean_text = clean_text.replace("I", "1")
      clean_text = clean_text.upper()
      text_list.append(clean_text)
    else:
      text_list.append(text)


In [ ]:
df_fgsm["text_pred"] = text_list


In [ ]:
ocr_model = gen_model('densenet_lite_136-gru', cnocr_model.rec_model._vocab)
mdape_rec = []
df_fgsm["text_pred"] = df_fgsm["text_pred"].fillna("")

for i in range(df_fgsm.shape[0]):
    text_pred = df_fgsm.at[i, 'text_pred']
    text_target = df_fgsm.at[i, 'plate_number']
    if len(text_pred) < 7:
      while len(text_pred) != 7:
          text_pred += "-"
    if len(text_pred) > 7:
      while len(text_pred) != 7:
        ind = random.randint(0, len(text_pred) - 1)
        text_pred = text_pred[:ind] + text_pred[ind + 1:]

    pred = ocr_model.compute_target([text_pred])
    target = ocr_model.compute_target([text_target])
    predictTensor = torch.tensor(pred[0].astype(float).tolist(), requires_grad=True)
    targetTensor = torch.tensor(target[0].astype(float).tolist(), requires_grad=False)
    tlist = targetTensor.tolist()[0]
    plist = predictTensor.tolist()[0]

    mdape_val = mdape(np.array(tlist), np.array(plist))
    mdape_rec.append(mdape_val)

In [ ]:
df_fgsm["rec_mdape"] = mdape_rec
df_fgsm["rec_mdape"] = df_fgsm["rec_mdape"].fillna(1.0)
df_fgsm.to_csv('fgsm_out.csv', index=False)


In [ ]:
df_fgsm["rec_mdape"].sum() / df_fgsm.shape[0] * 100

51.08850142747697

In [ ]:
!cp '/content/fgsm_out.csv' /content/drive/MyDrive/


##DI^2FGSM

In [ ]:
df_metadata = pd.read_csv('/content/drive/MyDrive/out.csv')


In [ ]:
df_difgsm = pd.read_csv('/content/drive/MyDrive/difgsm_out.csv')


In [ ]:
df_difgsm = df_metadata[["image_path", "x1_bbox", "y1_bbox", "x2_bbox", "y2_bbox", "plate_number"]].copy()
df_difgsm

,image_path,x1_bbox,y1_bbox,x2_bbox,y2_bbox,plate_number
0,/content/drive/MyDrive/CCPD2019_2K/ccpd_base/0...,360,479,546,543,皖AKJ718
1,/content/drive/MyDrive/CCPD2019_2K/ccpd_base/0...,208,474,414,579,皖AWC386
2,/content/drive/MyDrive/CCPD2019_2K/ccpd_base/0...,283,486,492,550,皖AV611U
3,/content/drive/MyDrive/CCPD2019_2K/ccpd_base/0...,272,512,481,574,皖AZ9X58
4,/content/drive/MyDrive/CCPD2019_2K/ccpd_base/0...,280,442,449,515,皖A0N621
...,...,...,...,...,...,...
1995,/content/drive/MyDrive/CCPD2019_2K/ccpd_blur/0...,306,494,408,531,皖A59S98
1996,/content/drive/MyDrive/CCPD2019_2K/ccpd_blur/0...,253,377,353,415,皖CR2366
1997,/content/drive/MyDrive/CCPD2019_2K/ccpd_blur/0...,329,554,505,619,皖A2S216
1998,/content/drive/MyDrive/CCPD2019_2K/ccpd_blur/0...,239,418,466,519,皖AC8Y96


In [ ]:
def align_predicted_text(txt):
      if len(txt) < 7:
          while len(txt) != 7:
              txt += "-"
      if len(txt) > 7:
        while len(txt) != 7:
          ind = random.randint(0, len(txt) - 1)
          txt = txt[:ind] + txt[ind + 1:]
      return txt


In [ ]:
def difgsm(img, epsilon, iterations, text_target, grad):
  loss = nn.MSELoss(reduction='mean')
  pertubed_img = fgsm(img, grad, epsilon)
  ocr_predict = cnocr_model.ocr(pertubed_img)

  text = "-------"
  if len(ocr_predict) > 0:
      text = ocr_predict[0]["text"]
      text = prepare_txt(text)

  if len(text) != 7:
    text = align_predicted_text(text)

  pred = ocr_model.compute_target([text])
  target = ocr_model.compute_target([text_target])
  predictTensor = torch.tensor(pred[0].astype(float).tolist(), requires_grad=True)
  targetTensor = torch.tensor(target[0].astype(float).tolist(), requires_grad=False)

  loss_res = loss(predictTensor, targetTensor)

  loss_res.backward()
  grad = predictTensor.grad

  for i in range(0, iterations):
    if random.randint(0, 100) <= 60:
      grad = torch.abs(grad)
      grad = grad / torch.mean(grad)
      grad = grad[0]

    if grad.shape[0] == 1:
      grad = grad[0]

    grad = grad.sign()

    pertubed_img = fgsm(img, grad, epsilon)
    ocr_predict = cnocr_model.ocr(pertubed_img)


    text = "-------"
    if len(ocr_predict) > 0:
        text = ocr_predict[0]["text"]
        text = prepare_txt(text)

    if len(text) != 7:
      text = align_predicted_text(text)

    pred = ocr_model.compute_target([text])

    predictTensor = torch.tensor(pred[0].astype(float).tolist(), requires_grad=True)
    targetTensor = torch.tensor(target[0].astype(float).tolist(), requires_grad=False)


    loss_res = loss(predictTensor, targetTensor)

    loss_res.backward()
    grad = predictTensor.grad

  if grad.shape[0] == 1:
      grad = grad[0]
  grad = grad.sign()
  pertubed_img = fgsm(img, grad, epsilon)


  return pertubed_img

In [ ]:
path = '/content/drive/MyDrive/CCPD_DIFGSM'
os.mkdir(path)
for i in list_sub:
  os.mkdir(f"{path}/{i}")

In [ ]:
import random

ocr_model = gen_model('densenet_lite_136-gru', cnocr_model.rec_model._vocab)
loss = nn.MSELoss(reduction='mean')
paths = []
epsilon = 0.02

for i in range(df_metadata.shape[0]):
  img_path = df_metadata.at[i, 'image_path']

  ori_img = cv2.imread(img_path)
  img = torch.from_numpy(ori_img)

  predict = [df_metadata.at[i, "x1_pred"], df_metadata.at[i, "y1_pred"], df_metadata.at[i, "x2_pred"], df_metadata.at[i, "y2_pred"]]
  mdape_score = df_metadata.at[i, 'det_mdape']
  if mdape_score != 1.0:
      target = df_metadata.at[i, 'plate_number']

      grad = df_metadata.at[i, 'grad']
      grad = df_metadata.at[i, 'grad'][1:-2].split(",")
      grad = [float(i) for i in grad]

      grad = torch.tensor(grad)
      crop_image = img[int(predict[1]):int(predict[3]), int(predict[0]):int(predict[2])]
      pertubed_image = difgsm(crop_image, epsilon, 20, target, grad)
      img = img.numpy()
      img[int(predict[1]):int(predict[3]), int(predict[0]):int(predict[2])] = pertubed_image

  if torch.is_tensor(img):
    img = img.numpy()

  img = Image.fromarray(img)
  img_path = img_path.replace("CCPD2019_2K", "CCPD_DIFGSM")
  paths.append(img_path)
  img.save(img_path)




In [ ]:
df_difgsm["image_path"] = paths

In [ ]:
df_difgsm.to_csv('difgsm_out.csv', index=False)


In [ ]:
!cp '/content/difgsm_out.csv' /content/drive/MyDrive/


In [ ]:
df_difgsm = pd.read_csv('/content/drive/MyDrive/difgsm_out.csv')


In [ ]:
df_metadata['det_mdape'] = df_metadata['det_mdape'].fillna(1.0)

In [ ]:
x1_pred = []
y1_pred = []
x2_pred = []
y2_pred = []
for i in range(df_metadata.shape[0]):
  path = df_difgsm.at[i, 'image_path']
  predict = [df_metadata.at[i, "x1_bbox"], df_metadata.at[i, "y1_bbox"], df_metadata.at[i, "x2_bbox"], df_metadata.at[i, "y2_bbox"]]
  mdape_score = df_metadata.at[i, 'det_mdape']

  if mdape_score != 1.0:
    results = model(path, size=640)
    results = model(path, augment=True)
    predictions = results.pred[0]

    try:
      boxes = predictions[:, :4].tolist()[0] # x1, y1, x2, y2
      x1_pred.append(boxes[0])
      y1_pred.append(boxes[1])
      x2_pred.append(boxes[2])
      y2_pred.append(boxes[3])

    except IndexError:
      x1_pred.append(None)
      y1_pred.append(None)
      x2_pred.append(None)
      y2_pred.append(None)
  else:
    x1_pred.append(None)
    y1_pred.append(None)
    x2_pred.append(None)
    y2_pred.append(None)







In [ ]:
df_difgsm["x1_pred"] = x1_pred
df_difgsm["y1_pred"] = y1_pred
df_difgsm["x2_pred"] = x2_pred
df_difgsm["y2_pred"] = y2_pred

In [ ]:
df_difgsm.to_csv('difgsm_out.csv', index=False)


In [ ]:
!cp '/content/difgsm_out.csv' /content/drive/MyDrive/


In [ ]:
mdape_list = []
for i in range (df_difgsm.shape[0]):
    target = np.array([df_difgsm.at[i, "x1_bbox"], df_difgsm.at[i, "y1_bbox"], df_difgsm.at[i, "x2_bbox"], df_difgsm.at[i, "y2_bbox"]])
    predict = np.array([df_difgsm.at[i, "x1_pred"], df_difgsm.at[i, "y1_pred"], df_difgsm.at[i, "x2_pred"], df_difgsm.at[i, "y2_pred"]])
    e = mdape(target, predict)
    mdape_list.append(e)

In [ ]:
df_difgsm["det_mdape"] = mdape_list
df_difgsm["det_mdape"] = df_difgsm["det_mdape"].fillna(1.0)

In [ ]:
df_difgsm["det_mdape"].sum() / df_difgsm.shape[0] * 100

7.648067321994781

In [ ]:
number_pred = []
for i in range (df_difgsm.shape[0]):
  img_path = df_difgsm.at[i, 'image_path']
  img = cv2.imread(img_path)

  predict = [df_difgsm.at[i, "x1_pred"], df_difgsm.at[i, "y1_pred"], df_difgsm.at[i, "x2_pred"], df_difgsm.at[i, "y2_pred"]]
  text = ""
  mdape_score = df_difgsm.at[i, 'det_mdape']
  if mdape_score != 1.0:
    crop_image = img[int(predict[1]):int(predict[3]), int(predict[0]):int(predict[2])]
    ocr_predict = cnocr_model.ocr(crop_image)
    if len(ocr_predict) > 0:
      text = ocr_predict[0]["text"]
  number_pred.append(text)

In [ ]:
df_difgsm["text_pred"] = number_pred


In [ ]:
text_list = []
for i in range(df_difgsm.shape[0]):
    text = df_difgsm.at[i, 'text_pred']
    if text != "":
      clean_text = re.sub(r"[ ·:-]", "", text)
      clean_text = clean_text.replace("O", "0")
      clean_text = clean_text.replace("I", "1")
      clean_text = clean_text.upper()
      text_list.append(clean_text)
    else:
      text_list.append(text)

In [ ]:
df_difgsm["text_pred"] = text_list


In [ ]:
ocr_model = gen_model('densenet_lite_136-gru', cnocr_model.rec_model._vocab)
mdape_rec = []
df_difgsm["text_pred"] = df_difgsm["text_pred"].fillna("")

for i in range(df_difgsm.shape[0]):
    text_pred = df_difgsm.at[i, 'text_pred']
    text_target = df_difgsm.at[i, 'plate_number']
    if len(text_pred) < 7:
      while len(text_pred) != 7:
          text_pred += "-"
    if len(text_pred) > 7:
      while len(text_pred) != 7:
        ind = random.randint(0, len(text_pred) - 1)
        text_pred = text_pred[:ind] + text_pred[ind + 1:]

    pred = ocr_model.compute_target([text_pred])
    target = ocr_model.compute_target([text_target])
    predictTensor = torch.tensor(pred[0].astype(float).tolist(), requires_grad=True)
    targetTensor = torch.tensor(target[0].astype(float).tolist(), requires_grad=False)
    tlist = targetTensor.tolist()[0]
    plist = predictTensor.tolist()[0]

    mdape_val = mdape(np.array(tlist), np.array(plist))
    mdape_rec.append(mdape_val)

In [ ]:
df_difgsm["rec_mdape"] = mdape_rec
df_difgsm["rec_mdape"] = df_difgsm["rec_mdape"].fillna(1.0)
df_difgsm.to_csv('difgsm_out.csv', index=False)

In [ ]:
!cp '/content/difgsm_out.csv' /content/drive/MyDrive/


In [ ]:
df_difgsm["rec_mdape"].sum() / df_difgsm.shape[0] * 100

54.26528697805119

#Defence

##Adversarial training based on PGD

In [ ]:
df_metadata = pd.read_csv('/content/drive/MyDrive/out.csv')


In [ ]:
path = '/content/drive/MyDrive/CCPD_PGD'
os.mkdir(path)
for i in list_sub:
  os.mkdir(f"{path}/{i}")

In [ ]:
def pgd(image, target_txt, eps=0.02, alpha=0.001, iters=5):
  loss = nn.MSELoss(reduction='mean')
  for i in range(iters):
      torchImage = torch.from_numpy(image)
      predicted_text = cnocr_model.ocr(image)

      text = "-------"
      if len(predicted_text) > 0:
          text = predicted_text[0]["text"]
          text = prepare_txt(text)

      if len(text) != 7:
          text = align_predict(text)
      pred = ocr_model.compute_target([text])
      target = ocr_model.compute_target([target_txt])
      predictTensor = torch.tensor(pred[0].astype(float).tolist(), requires_grad=True)
      targetTensor = torch.tensor(target[0].astype(float).tolist(), requires_grad=False)
      loss_res = loss(predictTensor, targetTensor)
      loss_res.backward()
      grad = predictTensor.grad.sign()[0]
      grad_shape = grad.shape[0]
      changed_grad = grad.view(1, 1, grad_shape).expand(image.shape[0], image.shape[1], grad_shape)
      changed_grad = nn.functional.interpolate(changed_grad, 3)

      adv = torchImage + alpha * changed_grad
      eta = torch.clamp(adv - torchImage, min=-eps, max=eps)
      image = torch.clamp(torchImage + eta, min=0, max=255).numpy()

  return image

In [ ]:
ocr_model = gen_model('densenet_lite_136-gru', cnocr_model.rec_model._vocab)
paths = []

for i in range(df_metadata.shape[0]):
  img_path = df_metadata.at[i, 'image_path']

  ori_img = cv2.imread(img_path)
  predict = [df_metadata.at[i, "x1_bbox"], df_metadata.at[i, "y1_bbox"], df_metadata.at[i, "x2_bbox"], df_metadata.at[i, "y2_bbox"]]
  crop_image = ori_img[int(predict[1]):int(predict[3]), int(predict[0]):int(predict[2])]
  target = df_metadata.at[i, 'plate_number']
  perturbed = pgd(crop_image, target)
  perturbed = (perturbed * 255).astype(np.uint8)
  perturbed = (perturbed * 255).astype(np.uint8)
  ori_img[int(predict[1]):int(predict[3]), int(predict[0]):int(predict[2])] = perturbed

  img = Image.fromarray(ori_img)
  img_path = img_path.replace("CCPD2019_2K", "CCPD_PGD")
  paths.append(img_path)
  img.save(img_path)




In [ ]:
df_pgd = df_metadata[["image_path", "plate_number"]].copy()


In [ ]:
df_pgd["image_path"] = paths

In [ ]:
df_pgd.to_csv('pgd_out.csv', index=False)
!cp '/content/pgd_out.csv' /content/drive/MyDrive/

In [ ]:
import shutil
shutil.copytree('/content/drive/MyDrive/CCPD_PGD', "./CCPD_PGD")

'./CCPD_PGD'

In [ ]:
df_pgd = pd.read_csv('/content/drive/MyDrive/pgd_out.csv')


In [ ]:
!mkdir "PGD_TRAINING"

In [ ]:
for i in list_sub:
  os.mkdir(f"/content/PGD_TRAINING/{i}")

In [ ]:
paths = []
plates = []
for i in range(df_pgd.shape[0]):
    predict = [df_metadata.at[i, "x1_pred"], df_metadata.at[i, "y1_pred"], df_metadata.at[i, "x2_pred"], df_metadata.at[i, "y2_pred"]]
    text = ""
    mdape_score = df_metadata.at[i, 'det_mdape']
    if mdape_score != 1.0:
      img_path = df_metadata.at[i, 'image_path']
      img = cv2.imread(img_path)
      crop_image = img[int(predict[1]):int(predict[3]), int(predict[0]):int(predict[2])]

      img_path = img_path.replace("/content/drive/MyDrive/CCPD2019_2K/", "")
      img = Image.fromarray(crop_image)
      img.save("/content/PGD_TRAINING/"+img_path)

      plate = df_pgd.at[i, 'plate_number']
      l = list(plate)
      l.insert(2, "•")
      plates.append(" ".join(l))

      # plates.append(plate)
      paths.append(img_path)



In [ ]:
df_training = pd.DataFrame()

In [ ]:
df_training['image_path'] = paths
df_training["plate_number"] = plates

In [ ]:
df_training

,image_path,plate_number
0,ccpd_base/0100754310345-90_90-360&479_546&543-...,皖 A • K J 7 1 8
1,ccpd_base/0148275862069-94_82-208&474_414&579-...,皖 A • W C 3 8 6
2,ccpd_base/0124329501916-90_88-283&486_492&550-...,皖 A • V 6 1 1 U
3,ccpd_base/0100754310345-90_86-272&512_481&574-...,皖 A • Z 9 X 5 8
4,ccpd_base/0127825670498-85_101-280&442_449&515...,皖 A • 0 N 6 2 1
...,...,...
1889,ccpd_blur/0045-1_3-306&494_408&531-408&531_307...,皖 A • 5 9 S 9 8
1890,ccpd_blur/0045-2_1-253&377_353&415-353&411_255...,皖 C • R 2 3 6 6
1891,ccpd_blur/0136-1_2-329&554_505&619-505&619_330...,皖 A • 2 S 2 1 6
1892,ccpd_blur/0274-1_1-239&418_466&519-466&519_239...,皖 A • C 8 Y 9 6


In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df_training, test_size=0.2)

In [ ]:
with open("train.tsv",'w') as write_tsv:
    write_tsv.write(train.to_csv(sep='\t', index=False, header=False))

In [ ]:
with open("dev.tsv",'w') as write_tsv:
    write_tsv.write(test.to_csv(sep='\t', index=False, header=False))

In [ ]:
!cp '/content/train.tsv' /content/drive/MyDrive/
!cp '/content/dev.tsv' /content/drive/MyDrive/

In [ ]:
!mkdir "train_data"

In [ ]:
!mv "/content/dev.tsv" "/content/train_data"
!mv "/content/train.tsv" "/content/train_data"

In [ ]:
!pip install torchaudio

In [ ]:
from CnOCR.cnocr.consts import CN_VOCAB_FP
from CnOCR.cnocr.data_utils.aug import NormalizeAug
from CnOCR.cnocr.dataset import OcrDataModule
from CnOCR.cnocr.trainer import PlTrainer

/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [ ]:
from torchvision import transforms


In [ ]:
train_transform = transforms.Compose([NormalizeAug()])
val_transform = NormalizeAug()

In [ ]:
INDEX_DIR = '/content/train_data'
EXAMPLE_DIR = '/content/PGD_TRAINING'

data_mod = OcrDataModule(
        index_dir=INDEX_DIR,
        vocab_fp=CN_VOCAB_FP,
        img_folder=EXAMPLE_DIR,
        train_transforms=train_transform,
        val_transforms=val_transform,
        batch_size=32,
        num_workers=0,
        pin_memory=False,
    )
config = {
     "devices": 1,
     "accelerator": "cpu",
        "epochs": 50,
        "batch_size": 32,
        "num_workers": 0,
        "pin_memory": "false",
        "optimizer": "adam",
        "learning_rate": 1e-3,
        "weight_decay": 1e-6,
        "metrics": {
            "complete_match": {},
            "cer": {}
        },
        "lr_scheduler": {
            "name": "cos_warmup",
            "min_lr_mult_factor": 0.01,
            "warmup_epochs": 0.2
        },
        "precision": 32,
        "limit_train_batches": 1.0,
        "limit_val_batches": 1.0,
        "pl_checkpoint_monitor": "val-complete_match-epoch",
        "pl_checkpoint_mode": "max"
    }
trainer = PlTrainer(config)
# model = gen_model('densenet_lite_136-gru', data_mod.vocab)
trainer.fit(cnocr_model.rec_model._model, datamodule=data_mod)
print(trainer.saved_model_file)


Map:   0%|          | 0/1515 [00:00<?, ? examples/s]

Map:   0%|          | 0/379 [00:00<?, ? examples/s]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:`Trainer(limit_train_batches=1.0)` was configured so 100% of the batches per epoch will be used..
INFO:pytorch_lightning.utilities.rank_zero:`Trainer(limit_val_batches=1.0)` was configured so 100% of the batches will be used..


INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type     | Params
-----------------------------------
0 | model | OcrModel | 3.1 M 
-----------------------------------
3.1 M     Trainable params
0         Non-trainable params
3.1 M     Total params
12.523    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=50` reached.


runs/CnOCR-Rec/5o3rn8xy/checkpoints/epoch=000-val-complete_match-epoch=0.0000-model.ckpt


##Verify on FGSM

In [ ]:
df_pgd_fgsm = pd.read_csv('/content/drive/MyDrive/fgsm_out.csv')


In [ ]:
number_pred = []
for i in range (df_pgd_fgsm.shape[0]):
  img_path = df_pgd_fgsm.at[i, 'image_path']
  img = cv2.imread(img_path)

  predict = [df_pgd_fgsm.at[i, "x1_pred"], df_pgd_fgsm.at[i, "y1_pred"], df_pgd_fgsm.at[i, "x2_pred"], df_pgd_fgsm.at[i, "y2_pred"]]
  text = ""
  mdape_score = df_pgd_fgsm.at[i, 'det_mdape']
  if mdape_score != 1.0:
    crop_image = img[int(predict[1]):int(predict[3]), int(predict[0]):int(predict[2])]
    ocr_predict = cnocr_model.ocr(crop_image)
    if len(ocr_predict) > 0:
      text = ocr_predict[0]["text"]
  number_pred.append(text)

In [ ]:
df_pgd_fgsm["text_pred"] = number_pred


In [ ]:
df_pgd_fgsm.to_csv('pgd_fgsm_out.csv', index=False)
!cp '/content/pgd_fgsm_out.csv' /content/drive/MyDrive/

In [ ]:
df_pgd_fgsm = pd.read_csv('/content/drive/MyDrive/pgd_fgsm_out.csv')


In [ ]:
ocr_model = gen_model('densenet_lite_136-gru', cnocr_model.rec_model._vocab)
mdape_rec = []
df_pgd_fgsm["text_pred"] = df_pgd_fgsm["text_pred"].fillna("")

for i in range(df_pgd_fgsm.shape[0]):
    text_pred = df_pgd_fgsm.at[i, 'text_pred']
    text_target = df_pgd_fgsm.at[i, 'plate_number']
    if len(text_pred) < 7:
      while len(text_pred) != 7:
          text_pred += "-"
    if len(text_pred) > 7:
      while len(text_pred) != 7:
        ind = random.randint(0, len(text_pred) - 1)
        text_pred = text_pred[:ind] + text_pred[ind + 1:]

    pred = ocr_model.compute_target([text_pred])
    target = ocr_model.compute_target([text_target])
    predictTensor = torch.tensor(pred[0].astype(float).tolist(), requires_grad=True)
    targetTensor = torch.tensor(target[0].astype(float).tolist(), requires_grad=False)
    tlist = targetTensor.tolist()[0]
    plist = predictTensor.tolist()[0]

    mdape_val = mdape(np.array(tlist), np.array(plist))
    mdape_rec.append(mdape_val)

In [ ]:
df_pgd_fgsm["rec_mdape"] = mdape_rec
df_pgd_fgsm["rec_mdape"] = df_pgd_fgsm["rec_mdape"].fillna(1.0)

In [ ]:
df_pgd_fgsm["rec_mdape"].sum() / df_pgd_fgsm.shape[0] * 100

53.65187263745973

In [ ]:
df_pgd_fgsm.to_csv('pgd_fgsm_out.csv', index=False)
!cp '/content/pgd_fgsm_out.csv' /content/drive/MyDrive/

##Verify on DIFGSM

In [ ]:
df_pgd_difgsm = pd.read_csv('/content/drive/MyDrive/fgsm_out.csv')


In [ ]:
number_pred = []
for i in range (df_pgd_difgsm.shape[0]):
  img_path = df_pgd_difgsm.at[i, 'image_path']
  img = cv2.imread(img_path)

  predict = [df_pgd_difgsm.at[i, "x1_pred"], df_pgd_difgsm.at[i, "y1_pred"], df_pgd_difgsm.at[i, "x2_pred"], df_pgd_difgsm.at[i, "y2_pred"]]
  text = ""
  mdape_score = df_pgd_difgsm.at[i, 'det_mdape']
  if mdape_score != 1.0:
    crop_image = img[int(predict[1]):int(predict[3]), int(predict[0]):int(predict[2])]
    ocr_predict = cnocr_model.ocr(crop_image)
    if len(ocr_predict) > 0:
      text = ocr_predict[0]["text"]
  number_pred.append(text)

In [ ]:
df_pgd_difgsm["text_pred"] = number_pred


In [ ]:
df_pgd_difgsm.to_csv('pgd_difgsm_out.csv', index=False)
!cp '/content/pgd_difgsm_out.csv' /content/drive/MyDrive/

In [ ]:
df_pgd_difgsm = pd.read_csv('/content/drive/MyDrive/pgd_difgsm_out.csv')


In [ ]:
ocr_model = gen_model('densenet_lite_136-gru', cnocr_model.rec_model._vocab)
mdape_rec = []
df_pgd_difgsm["text_pred"] = df_pgd_difgsm["text_pred"].fillna("")

for i in range(df_pgd_difgsm.shape[0]):
    text_pred = df_pgd_difgsm.at[i, 'text_pred'].replace(" ", "-")
    text_target = df_pgd_difgsm.at[i, 'plate_number']
    if len(text_pred) < 7:
      while len(text_pred) != 7:
          text_pred += "-"
    if len(text_pred) > 7:
      while len(text_pred) != 7:
        ind = random.randint(0, len(text_pred) - 1)
        text_pred = text_pred[:ind] + text_pred[ind + 1:]


    pred = ocr_model.compute_target([text_pred])
    target = ocr_model.compute_target([text_target])
    predictTensor = torch.tensor(pred[0].astype(float).tolist(), requires_grad=True)
    targetTensor = torch.tensor(target[0].astype(float).tolist(), requires_grad=False)
    tlist = targetTensor.tolist()[0]
    plist = predictTensor.tolist()[0]

    mdape_val = mdape(np.array(tlist), np.array(plist))
    mdape_rec.append(mdape_val)

In [ ]:
df_pgd_difgsm["rec_mdape"] = mdape_rec
df_pgd_difgsm["rec_mdape"] = df_pgd_difgsm["rec_mdape"].fillna(1.0)

In [ ]:
df_pgd_difgsm["rec_mdape"].sum() / df_pgd_difgsm.shape[0] * 100

116.7125448591344

In [ ]:
df_pgd_difgsm.to_csv('pgd_difgsm_out.csv', index=False)
!cp '/content/pgd_difgsm_out.csv' /content/drive/MyDrive/